In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

In [16]:
flight_itinerary_df = pd.read_csv("Resources/itineraries.csv", nrows=1000000)
flight_itinerary_df.head()


,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H29M,0,False,False,...,1650223560,2022-04-17T15:26:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,8940,947,coach
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,1650200400,2022-04-17T09:00:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9000,947,coach
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,1650218700,2022-04-17T14:05:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757-200,9000,947,coach
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H32M,0,False,False,...,1650227460,2022-04-17T16:31:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9120,947,coach
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H34M,0,False,False,...,1650213180,2022-04-17T12:33:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9240,947,coach


In [30]:
flight_itinerary_df.columns
flight_itinerary_df["legId"].nunique()
dupes = flight_itinerary_df[flight_itinerary_df["legId"].duplicated()]["legId"].unique()

dupes

array(['ef4b6a406cdfaa38a1c1d4911c1074de',
       'ad9a05a90b4ef222054f4059cc6bfdc7',
       '2c01653b9f36866864df5e5e22961b59', ...,
       'de6456e27327c6698afe5853739221d3',
       'a1d867a12c631f41562ec88e4fbe9f1c',
       'b3972452c388f0f3222177e0fc084f47'], dtype=object)

In [34]:
temp = flight_itinerary_df[(flight_itinerary_df["legId"] == "ef4b6a406cdfaa38a1c1d4911c1074de")]
temp.head()
temp.tail()


,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
8259,ef4b6a406cdfaa38a1c1d4911c1074de,2022-04-16,2022-04-18,ATL,BOS,LA0NX0MC,PT2H32M,0,False,False,...,1650340620,2022-04-18T23:57:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757,9120,947,coach
87657,ef4b6a406cdfaa38a1c1d4911c1074de,2022-04-17,2022-04-18,ATL,BOS,LA0NX0MC,PT2H32M,0,False,False,...,1650340620,2022-04-18T23:57:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757,9120,947,coach


In [36]:
airline_df = pd.read_csv("Resources/Airline_Delay_cause.csv")
airline_df.head(20)


Index(['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name',
       'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct',
       'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted',
       'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay'],
      dtype='object')

In [51]:

airline_df["carrier_name"].unique()

array(['Endeavor Air Inc.', 'American Airlines Inc.',
       'Alaska Airlines Inc.', 'JetBlue Airways', 'Delta Air Lines Inc.',
       'Frontier Airlines Inc.', 'Allegiant Air',
       'Hawaiian Airlines Inc.', 'Envoy Air', 'Spirit Air Lines',
       'PSA Airlines Inc.', 'SkyWest Airlines Inc.', 'Horizon Air',
       'United Air Lines Inc.', 'Southwest Airlines Co.',
       'Mesa Airlines Inc.', 'Republic Airline',
       'ExpressJet Airlines LLC', 'ExpressJet Airlines Inc.',
       'Virgin America', 'US Airways Inc.', 'AirTran Airways Corporation',
       'American Eagle Airlines Inc.', 'Pinnacle Airlines Inc.',
       'Atlantic Southeast Airlines', 'Continental Air Lines Inc.',
       'Comair Inc.', 'Northwest Airlines Inc.', 'Aloha Airlines Inc.',
       'ATA Airlines d/b/a ATA', 'Independence Air',
       'America West Airlines Inc.', 'Atlantic Coast Airlines'],
      dtype=object)